In [1]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [2]:
print(twenty_train.target_names)
print(twenty_train.data[0])
print(twenty_train.target[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have o

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf' , TfidfTransformer()),
                    ('clf' , MultinomialNB())])

In [7]:
MultinomialNB? # Alpha값이 라플라스 스무딩값..

In [8]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [9]:
import numpy as np

twenty_test= fetch_20newsgroups(subset = 'test', shuffle = True)
predicted = text_clf.predict(twenty_test.data)
# 예측값이 나왔으니, 비교를 위해
np.mean(predicted == twenty_test.target)

0.7738980350504514

In [10]:
predicted == twenty_test.target

array([ True, False,  True, ...,  True, False,  True])

In [11]:
# sklearn의 장점은 하이퍼 파라미터를 조정을 통해 좋은 모델을 만들어버릴 수 있다
from sklearn.model_selection import GridSearchCV
parameters_clf = {'vect__ngram_range' : [(1,1), (1,2)],
                  'tfidf__use_idf': (True, False),
                  'clf__alpha' : (1,0.1,0.01,0.001,0.0001)        # 이건 그냥 규칙!, 던더!, Pipeline 안에 있는 
                  }                                               # 함수들의 param 부를 때, 같은이름__하이퍼파라미터 ㅇ;렇게

gs_clf = GridSearchCV(text_clf, parameters_clf, n_jobs = -1, verbose = 2)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
print('best_score : {}'.format(gs_clf.best_score_))
best_parameters = gs_clf.best_estimators_.get_params()
for param_name in sorted(list(best_parameters.keys())):
    print("\t{} : {}".format(param_name, best_parameters[param_name]))


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  8.1min finished


best_score : 0.9157684864695698


AttributeError: 'GridSearchCV' object has no attribute 'best_parameters_'

In [13]:
print('best_score : {}'.format(gs_clf.best_score_))
best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(list(best_parameters.keys())):
    print("\t{} : {}".format(param_name, best_parameters[param_name]))

best_score : 0.9157684864695698
	clf : MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)
	clf__alpha : 0.001
	clf__class_prior : None
	clf__fit_prior : True
	memory : None
	steps : [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True))]
	tfidf : TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)
	tfidf__norm : l2
	tfidf__smooth_idf : True
	tfidf__sublinear_tf : False
	tfidf__use_idf : True
	vect : CountVectorizer(analyzer='w

In [14]:
predicted = gs_clf.best_estimator_.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.8361656930430165